In [64]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu
!pip install PyPDF2

In [65]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
import PyPDF2
warnings.filterwarnings("ignore")

In [ ]:
openai.api_key = " "
os.environ['OPENAI_API_KEY'] = ' '

In [67]:
def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text()
    return text

In [68]:
pdf_text = extract_text_from_pdf('/content/financial-wellness-journal-english.pdf')

In [69]:
print(pdf_text)

Financial Wellness Journal 
Introduction
Getting started Pre-assessmentTABLE OF CONTENTS
Chapter 1: All About Financial Wellness
Chapter 
introductionHow FinEd can 
help you achieve 
financial wellnessThe life stages
of money Facing these 
difficulties head onActivity: Visioning 
exercise
Chapter 2: All About Saving
Case study: Mang 
Rafael’s saving 
problem Common 
misconceptions 
about savingWhat are the goals 
of saving?What are the tools 
we can use to start 
saving?Where do we save?
Case study: Aling 
Yolanda insures her 
daughter’s future What is insurance? Why do we need 
to insure ourselves 
and our family?What are the 
different kinds
of insurance?Chapter wrap-up
Chapter 4: All About CreditChapter 3: All About Insurance
Chapter 5: All About Investing
Conclusion3 4
6 7 8 9 11
12 13 14 15 20
23 21 24
27 28 29 29 31
33 34 35 36
38 39 39 4037
41 42 43 43 46Where should our 
savings go?What are the 
methods of 
budgeting?Chapter wrap-up
Case study: 
Teacher Mary is 
finally debt-fr

In [136]:
chunk_size = 5000
chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]

In [137]:
embeddings = [get_embedding(chunk, engine='text-embedding-3-small') for chunk in chunks]

In [138]:
embedding_dim = len(embeddings[0])

In [139]:
embedding_np = np.array(embeddings).astype('float32')

In [140]:
index = faiss.IndexFlatL2(embedding_dim)

In [141]:
index.add(embedding_np)

In [142]:
user_message = "What are the different kinds of insurances?"

In [143]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')

In [144]:
_, indices = index.search(query_embedding_np, 5)

In [145]:
retrieved_docs = [chunks[i] for i in indices[0]]

In [146]:
context = ' '.join(retrieved_docs)

In [147]:
structured_prompt = f'Context: \n{context} \n\n Query: {user_message} \n\n Response:'

In [148]:
print(structured_prompt)

Context: 
product that 
comes in three options depending on your comes in three options depending on your 
budget and length of time you wish to be budget and length of time you wish to be 
covered by the policy: one year, five years, covered by the policy: one year, five years, 
or ten years.or ten years.
5BPI-PhilamLife Protect Insurance. BPI-Philam Life  Assurance Corp.
https://www.bpi-philam.com/content/dam/bplac/en/docs/brochures/Life%20Protect.pdfCHAPTER 3: ALL ABOUT INSURANCE
30TYPE #2:
NON-LIFE 
INSURANCENon-life insurance helps you cover for Non-life insurance helps you cover for 
possible loss of or damage to property. possible loss of or damage to property. 
Shelling out huge sums of money for Shelling out huge sums of money for 
car repair, for example, can set you back car repair, for example, can set you back 
financially, especially if you are reliant on financially, especially if you are reliant on 
your car! your car! 
Non-life insurance softens the blow of Non-life in

In [149]:
struct = [{"role": "system", "content": "You are a helpful assistant"}]

In [150]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [151]:
print(response)

The different kinds of insurance can be categorized into several types, each serving distinct purposes. Here are the main types mentioned:

1. **Life Insurance**: 
   - This type of insurance provides financial assistance to your family or beneficiaries in the event of your death or an accident. It ensures that they are financially supported during a difficult time. An example is the BPI-Philam Life Protect Insurance, which offers coverage starting at Php 320 per month.

2. **Non-Life Insurance**: 
   - This category covers potential loss or damage to property rather than life. Examples include:
     - **Fire Insurance**: Protects against losses from fire and can also cover damages from other events like theft, natural disasters (earthquakes, floods), and riots. It can apply to residential homes, commercial buildings, and their contents.

3. **Pre-Need Plans**: 
   - These plans help individuals prepare for future expenses, such as education or funeral costs. Memorial plans are a commo